In [ ]:
# plot_uvlf.py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Base directories
base_data_dir = "/disk/xray15/aem2/data/28pams/IllustrisTNG/1P"
base_plot_dir = "/disk/xray15/aem2/plots/28pams/IllustrisTNG/1P"
param_info_file = "/disk/xray15/aem2/data/28pams/Info_IllustrisTNG_L25n256_28params.txt"

# UVLF specific directories
lf_data_dir = os.path.join(base_data_dir, "LFs/GALEX")
lf_plot_dir = {
    "parameter_variation": os.path.join(base_plot_dir, "LFs/parameter_variation"),
    "redshift_variation": os.path.join(base_plot_dir, "LFs/redshift_variation")
}

# Create plot directories
for dir_path in lf_plot_dir.values():
    os.makedirs(dir_path, exist_ok=True)

# Redshift mappings
redshift_values = {
    '044': {'redshift': 2.00, 'label': 'z2.0'},
    '052': {'redshift': 1.48, 'label': 'z1.5'},
    '060': {'redshift': 1.05, 'label': 'z1.0'},
    '086': {'redshift': 0.10, 'label': 'z0.1'}
}

def calculate_parameter_values(param_info):
    """Calculate all parameter values based on LogFlag."""
    min_val = param_info['MinVal']
    max_val = param_info['MaxVal']
    fid_val = param_info['FiducialVal']
    
    if param_info['LogFlag']:
        log_min = np.log10(min_val)
        log_max = np.log10(max_val)
        log_fid = np.log10(fid_val)
        n1_val = 10**(log_min + (log_fid - log_min)/2)
        val_1 = 10**(log_fid + (log_max - log_fid)/2)
    else:
        n1_val = min_val + (fid_val - min_val)/2
        val_1 = fid_val + (max_val - fid_val)/2
    
    return {
        'n2': min_val,
        'n1': n1_val,
        '1': val_1,
        '2': max_val
    }

def read_uvlf_data(filename):
    """Read UVLF data from file."""
    try:
        df = pd.read_csv(filename, sep='\t')
        return df['magnitude'].values, df['phi'].values, df['phi_sigma'].values, df['hist'].values
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None, None, None, None

def plot_parameter_variation_uvlf(param_num, snap, param_info, spec_type="intrinsic"):
    """Create UVLF plot showing parameter variations at fixed redshift."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    variations = ['n2', 'n1', '1', '2']
    colors = ['blue', 'green', 'red', 'purple']
    param_values = calculate_parameter_values(param_info)
    
    plotted_any = False
    
    for var, color in zip(variations, colors):
        filename = os.path.join(lf_data_dir, 
                              redshift_values[snap]['label'],
                              f"UVLF_{1}P_p{param_num}_{var}_{redshift_values[snap]['label']}_{spec_type}.txt")
        
        if os.path.exists(filename):
            mag, phi, phi_sigma, hist = read_uvlf_data(filename)
            if mag is not None:
                label = f'{param_info["ParamName"]} = {param_values[var]:.3g}'
                ax.errorbar(mag, 10**phi, yerr=10**phi_sigma, 
                          fmt='o-', color=color, label=label,
                          markersize=5, capsize=3)
                plotted_any = True
        else:
            print(f"File not found: {filename}")
    
    if plotted_any:
        ax.set_yscale('log')
        ax.set_xlabel('Absolute Magnitude (AB)', fontsize=12)
        ax.set_ylabel('Number Density (Mpc$^{-3}$ mag$^{-1}$)', fontsize=12)
        ax.set_title(f'GALEX FUV Luminosity Function\n{param_info["ParamName"]} (p{param_num}) at z = {redshift_values[snap]["redshift"]}')
        ax.grid(True, which='both', linestyle='--', alpha=0.6)
        ax.legend(loc='upper left', fontsize=10)
        
        plt.savefig(os.path.join(lf_plot_dir["parameter_variation"], 
                                f'UVLF_param{param_num}_{redshift_values[snap]["label"]}_{spec_type}.pdf'),
                    bbox_inches='tight', dpi=300)
    
    plt.close()

def plot_redshift_variation_uvlf(param_num, var, param_info, spec_type="intrinsic"):
    """Create UVLF plot showing redshift variations for a fixed parameter variation."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = plt.cm.viridis(np.linspace(0, 1, len(redshift_values)))
    plotted_any = False
    
    for (snap, z_info), color in zip(redshift_values.items(), colors):
        filename = os.path.join(lf_data_dir,
                              z_info['label'],
                              f"UVLF_1P_p{param_num}_{var}_{z_info['label']}_{spec_type}.txt")
        
        if os.path.exists(filename):
            mag, phi, phi_sigma, hist = read_uvlf_data(filename)
            if mag is not None:
                ax.errorbar(mag, 10**phi, yerr=10**phi_sigma, 
                          fmt='o-', color=color, label=f'z = {z_info["redshift"]:.2f}',
                          markersize=5, capsize=3)
                plotted_any = True
        else:
            print(f"File not found: {filename}")
    
    if plotted_any:
        ax.set_yscale('log')
        ax.set_xlabel('Absolute Magnitude (AB)', fontsize=12)
        ax.set_ylabel('Number Density (Mpc$^{-3}$ mag$^{-1}$)', fontsize=12)
        ax.set_title(f'GALEX FUV Luminosity Function\n{param_info["ParamName"]} variation {var}')
        ax.grid(True, which='both', linestyle='--', alpha=0.6)
        ax.legend(loc='upper left', fontsize=10)
        
        plt.savefig(os.path.join(lf_plot_dir["redshift_variation"], 
                                f'UVLF_param{param_num}_{var}_{spec_type}.pdf'),
                    bbox_inches='tight', dpi=300)
    
    plt.close()


In [ ]:

def main():
    # Read parameter information
    params_df = pd.read_csv(param_info_file)
    
    # Process parameters 1-28
    for param_num in range(1, 29):
        param_info = params_df.iloc[param_num-1]
        print(f"Processing parameter {param_num}")
        
        # Parameter variation plots at each redshift
        for snap in redshift_values:
            print(f"  Processing snapshot {snap}")
            for spec_type in ["intrinsic", "attenuated"]:
                plot_parameter_variation_uvlf(param_num, snap, param_info, spec_type)
        
        # Redshift variation plots for each parameter variation
        for var in ['n2', 'n1', '1', '2']:
            for spec_type in ["intrinsic", "attenuated"]:
                plot_redshift_variation_uvlf(param_num, var, param_info, spec_type)

if __name__ == "__main__":
    main()